In [53]:
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import urllib.request
import os
from google_patent_scraper import scraper_class
import json
import re
import timeit

In [78]:
ids = pd.read_csv(r".\gp-query2-2.csv",skipinitialspace=True)
print(ids.isnull().sum())
codes = [p[p.find('patent/')+7:-3] for p in ids['result link']]
ids['code'] = codes
ids.set_index(ids.code,inplace=True)
ids['citations'] = [[] for i in ids.index]
ids 

id                                0
title                             0
assignee                         15
inventor/author                 383
priority date                   324
filing/creation date              6
publication date                  1
grant date                    13623
result link                       0
representative figure link    10002
dtype: int64


,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,code,citations
code,,,,,,,,,,,,
KR100866792B1,KR-100866792-B1,Method and apparatus for generating face descr...,삼성전자주식회사,"문영수, 박규태, 자오지아리, 황산성, 황원준",2007-01-10,2007-01-10,2008-11-04,2008-11-04,https://patents.google.com/patent/KR100866792B...,https://patentimages.storage.googleapis.com/b5...,KR100866792B1,[]
KR100703693B1,KR-100703693-B1,System and method for face recognition,삼성전자주식회사,"김정배, 이종하",2005-01-13,2005-01-13,2007-04-05,2007-04-05,https://patents.google.com/patent/KR100703693B...,https://patentimages.storage.googleapis.com/5a...,KR100703693B1,[]
EP1732028B1,EP-1732028-B1,System and method for detecting an eye,"Delphi Technologies, Inc.","Riad I. Hammoud, Andrew L. Wilhelm",2005-06-10,2006-05-31,2012-01-11,2012-01-11,https://patents.google.com/patent/EP1732028B1/en,https://patentimages.storage.googleapis.com/88...,EP1732028B1,[]
US9396319B2,US-9396319-B2,Method of criminal profiling and person identi...,"Laird H. Shuart, Marcia L Shuart, Sharon E Jan...","Laird H. Shuart, Marcia L Shuart, Sharon E Jan...",2013-09-30,2014-09-24,2016-07-19,2016-07-19,https://patents.google.com/patent/US9396319B2/en,https://patentimages.storage.googleapis.com/ff...,US9396319B2,[]
KR100846500B1,KR-100846500-B1,Method and apparatus for recognizing face usin...,삼성전자주식회사,"기석철, 문영수, 박규태, 이종하, 황산성, 황원준",2006-11-08,2006-11-08,2008-07-17,2008-07-17,https://patents.google.com/patent/KR100846500B...,https://patentimages.storage.googleapis.com/c9...,KR100846500B1,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
WO2007077263A1,WO-2007077263-A1,Method for detecting chemical species and devi...,"Ramem, S.A.","Emilio Ramiro Arcas, Angel RIVERO JIMÉNEZ",2005-12-30,2005-12-30,2007-07-12,NaN,https://patents.google.com/patent/WO2007077263...,NaN,WO2007077263A1,[]
JPH05233978A,JP-H05233978-A,Fire detection system,"Kokusai Gijutsu Kaihatsu Kk, 国際技術開発株式会社","Masanori Hirasawa, 正憲 平澤",1992-02-25,1992-02-25,1993-09-10,NaN,https://patents.google.com/patent/JPH05233978A/en,NaN,JPH05233978A,[]
CN112307080A,CN-112307080-A,Power grid power supply loss analysis method d...,"国网福建省电力有限公司莆田供电公司, 国网福建省电力有限公司, 厦门亿力吉奥信息科技有限公司","陈晶腾, 魏海斌, 吴敏辉, 林培聪, 周天华, 陈辉河, 蒋雷震, 林宇澄, 陈芳, 林立...",NaN,2020-10-15,2021-02-02,NaN,https://patents.google.com/patent/CN112307080A/en,NaN,CN112307080A,[]


In [80]:
start = timeit.default_timer()  
for i in ids.index: 
    
    #request url
    fp = urllib.request.urlopen("https://patents.google.com/patent/"+i+"/en")
    #print("https://patents.google.com/patent/"+i+"/en")
    soup = BeautifulSoup(fp.read(), 'html')
    fp.close()
    
    #Get abstract
    abstract = soup.find('abstract')
    if abstract != None: ids.loc[i,'abstract'] = abstract.text
   
    #Get connections to other patents on google patents
    links = []
    trs = soup.find_all('tr',itemprop='similarDocuments')
    if len(trs)>=0:
        for tr in trs:
            l=tr.find('a',href=True)
            if l['href'] != '/patent/US20190146965A1/en' and l['href'].find('/patent/') !=-1 and l['href'].find('/en') !=-1 :
                links.append(l['href'][l['href'].find('/patent/')+8:-3])
    ids.loc[i,'citations'] = list(set(links))
    
    #Get code
    codes = soup.find_all('span',itemprop='Code')
    if len(codes)>=3:
        ids.loc[i,'class'] = codes[3].get_text()[:6]
    
    
    
   
stop = timeit.default_timer()    
print('Time: ', stop - start) 

URLError: <urlopen error [WinError 10060] Impossibile stabilire la connessione. Risposta non corretta della parte connessa dopo l'intervallo di tempo oppure mancata risposta dall'host collegato>

In [89]:
ids.to_csv("gp-query2-plus-2.csv")

In [52]:
ids['class'].value_counts()

G06F16    14
G06Q50    12
G06F40    10
H04L63     6
H04N21     5
G06Q30     4
G06K9/     3
G06F3/     3
G06F21     3
G06N5/     2
G06F8/     2
H04L9/     2
G06Q20     2
G06Q10     2
G06Q40     2
G06T1/     1
G06F11     1
G09B5/     1
G06N20     1
H04L41     1
H04L65     1
G06T5/     1
G06N3/     1
G08B5/     1
G10L15     1
H04L67     1
G06F       1
G16H20     1
A61B5/     1
Name: class, dtype: int64

In [72]:
ids

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,code,citations,abstract,class
code,,,,,,,,,,,,,,
KR100866792B1,KR-100866792-B1,Method and apparatus for generating face descr...,삼성전자주식회사,"문영수, 박규태, 자오지아리, 황산성, 황원준",2007-01-10,2007-01-10,2008-11-04,2008-11-04,https://patents.google.com/patent/KR100866792B...,https://patentimages.storage.googleapis.com/b5...,KR100866792B1,"[US20150178547A1, KR100707195B1, EP1716519A2, ...",\n\n 본 발명은 확장 국부 이진 패턴(Extended Local Binary P...,G06K9/
KR100703693B1,KR-100703693-B1,System and method for face recognition,삼성전자주식회사,"김정배, 이종하",2005-01-13,2005-01-13,2007-04-05,2007-04-05,https://patents.google.com/patent/KR100703693B...,https://patentimages.storage.googleapis.com/5a...,KR100703693B1,"[US10728242B2, US8433922B2, US9122913B2, US715...","\n\n 본 발명은 얼굴 인식에 관한 것으로서, 본 발명의 실시에 따른 얼굴 인식 ...",G06K9/
EP1732028B1,EP-1732028-B1,System and method for detecting an eye,"Delphi Technologies, Inc.","Riad I. Hammoud, Andrew L. Wilhelm",2005-06-10,2006-05-31,2012-01-11,2012-01-11,https://patents.google.com/patent/EP1732028B1/en,https://patentimages.storage.googleapis.com/88...,EP1732028B1,"[EP0989517B1, US7206435B2, US6028949A, JP31785...",NaN,G06K9/
US9396319B2,US-9396319-B2,Method of criminal profiling and person identi...,"Laird H. Shuart, Marcia L Shuart, Sharon E Jan...","Laird H. Shuart, Marcia L Shuart, Sharon E Jan...",2013-09-30,2014-09-24,2016-07-19,2016-07-19,https://patents.google.com/patent/US9396319B2/en,https://patentimages.storage.googleapis.com/ff...,US9396319B2,[],\nA method of criminal profiling and person id...,G06F21
KR100846500B1,KR-100846500-B1,Method and apparatus for recognizing face usin...,삼성전자주식회사,"기석철, 문영수, 박규태, 이종하, 황산성, 황원준",2006-11-08,2006-11-08,2008-07-17,2008-07-17,https://patents.google.com/patent/KR100846500B...,https://patentimages.storage.googleapis.com/c9...,KR100846500B1,"[US10565433B2, US20110135166A1]",\n\n 본 발명은 확장된 가보 웨이브렛 특징 들(Gabor Wavelet Feat...,G06K9/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WO2007077263A1,WO-2007077263-A1,Method for detecting chemical species and devi...,"Ramem, S.A.","Emilio Ramiro Arcas, Angel RIVERO JIMÉNEZ",2005-12-30,2005-12-30,2007-07-12,NaN,https://patents.google.com/patent/WO2007077263...,NaN,WO2007077263A1,[],NaN,NaN
JPH05233978A,JP-H05233978-A,Fire detection system,"Kokusai Gijutsu Kaihatsu Kk, 国際技術開発株式会社","Masanori Hirasawa, 正憲 平澤",1992-02-25,1992-02-25,1993-09-10,NaN,https://patents.google.com/patent/JPH05233978A/en,NaN,JPH05233978A,[],NaN,NaN
CN112307080A,CN-112307080-A,Power grid power supply loss analysis method d...,"国网福建省电力有限公司莆田供电公司, 国网福建省电力有限公司, 厦门亿力吉奥信息科技有限公司","陈晶腾, 魏海斌, 吴敏辉, 林培聪, 周天华, 陈辉河, 蒋雷震, 林宇澄, 陈芳, 林立...",NaN,2020-10-15,2021-02-02,NaN,https://patents.google.com/patent/CN112307080A/en,NaN,CN112307080A,[],NaN,NaN
